# Amazon Bedrock AgentCore 코드 인터프리터에서 명령 실행 - 튜토리얼

이 튜토리얼에서는 Amazon Bedrock AgentCore 코드 인터프리터를 사용하여 명령(셸 및 AWS CLI)을 실행하는 방법을 보여줍니다. AWS 서비스와 상호 작용하며, 특히 S3 작업에 중점을 둡니다. 다음 단계를 살펴보겠습니다.

1. 코드 인터프리터 생성
2. 코드 인터프리터 세션 시작
3. 명령 실행(셸 및 AWS CLI)
5. S3 작업 수행(버킷 생성, 객체 복사, 버킷 객체 나열)
6. 정리(세션 중지 및 코드 인터프리터 삭제)

## 필수 조건
- Bedrock AgentCore 코드 인터프리터 액세스 권한이 있는 AWS 계정
- 코드 인터프리터 리소스를 생성하고 관리하는 데 필요한 IAM 권한이 있어야 합니다.
- S3 작업을 수행하는 데 필요한 IAM 권한이 있어야 합니다.
- 필요한 Python 패키지(boto3 및 bedrock-agentcore 포함)가 설치되어 있어야 합니다.

## IAM 실행 역할에 다음 IAM 정책이 연결되어 있어야 합니다.

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

#### IAM 실행 역할에는 다음 신뢰 정책도 추가해야 합니다. 신뢰 정책에 bedrock-agentcore.amazonaws.com을 포함하면 Bedrock Agent 서비스 자체가 이 IAM 역할을 맡아 사용자를 대신하여 작업을 수행할 수 있습니다(예: Amazon S3).

```{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::<account_id>:root",
                "Service": [
                    "bedrock-agentcore.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole",
            "Condition": {}
        }
    ]
}

또한 이 튜토리얼에 설명된 S3 작업을 수행하려면 AmazonS3FullAccess IAM 정책을 IAM 실행 역할에 연결해야 합니다.

## 작동 방식

코드 실행 샌드박스는 코드 인터프리터, 셸, 파일 시스템을 갖춘 격리된 환경을 생성하여 에이전트가 사용자 쿼리를 안전하게 처리할 수 있도록 합니다. 대규모 언어 모델(LLM)을 통해 도구 선택을 돕고, 코드는 이 세션 내에서 실행된 후 합성을 위해 사용자 또는 에이전트에게 반환됩니다.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
import json
import boto3
from bedrock_agentcore.tools.code_interpreter_client import code_session, CodeInterpreter
from bedrock_agentcore._utils import endpoints
import time
from typing import Dict, Any, List

## 2. 구성 변수

코드 인터프리터 및 S3 작업에 필요한 구성 변수를 설정합니다. 또한 코드 인터프리터에 전달할 IAM 실행 역할도 제공하여 다른 AWS 리소스에 액세스할 수 있도록 합니다. 이 역할에는 위에서 설명한 대로 S3 권한이 필요합니다.

In [ ]:
# Configuration setup
execution_role_arn = "<execution-role-arn>"
unique_bucket_name = f"amzn-bucket-{int(time.time())}"
s3_path = f"s3://{unique_bucket_name}"

region = "us-west-2"

#local file that we will upload to Code interpreter and then to an S3 bucket
local_file = "samples/stats.py"

## 3. 엔드포인트 설정

boto3 클라이언트를 생성하려면 데이터 플레인과 제어 플레인 엔드포인트를 모두 구성해야 합니다.

In [ ]:
# Configure endpoints
data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

## 4. AWS 클라이언트 생성

제어 플레인과 데이터 플레인 작업 모두에 대해 boto3 클라이언트를 초기화합니다.

In [ ]:
# Create boto3 clients
cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

## 5. 코드 인터프리터 생성

특정 구성 매개변수를 사용하여 코드 인터프리터 인스턴스를 생성합니다.

코드 인터프리터를 구성할 때 네트워크 설정(샌드박스, 퍼블릭 또는 VPC), 종속성 구성, 보안 설정 및 코드 인터프리터가 액세스할 수 있는 AWS 리소스를 정의하는 IAM 런타임 역할을 통해 권한을 선택할 수 있습니다.

In [ ]:
# Create code interpreter
unique_name = f"s3InteractionEnv_{int(time.time())}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for S3 file operations",
    executionRoleArn=execution_role_arn,
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

## 6. Starting a Session

Create a session within the code interpreter to execute code.

In [ ]:
# Start session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="s3InteractionSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

## 7. Helper Function for Tool Execution

Define a utility function to simplify code interpreter tool invocation

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    response = dp_client.invoke_code_interpreter(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    for event in response["stream"]:
        return json.dumps(event["result"], indent=2)

## 8. Test Code Execution

### 8.1 Test the code interpreter with a simple Hello World example.

In [ ]:
# Test code execution
# S3 Operations
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "echo 'Hello World'"})
print(f"command result: {command_response}")

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

### 8.2 Next, lets install boto3 using PIP in to the sandbox

In [ ]:
# Test code execution
# S3 Operations
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "pip install boto3"})

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

## 9. File Operations and S3 Interaction by runninng commands

#### 9.1 Write local file to sandbox

In [ ]:
# Write file to sandbox
print("Writing file to sandbox")
try:
    with open(local_file, 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

files_to_create = [{
        "path": "stats.py",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"write files result: {write_files_response}")

#### 9.2 Create a S3 Bucket via code interpreter

In [ ]:
# S3 Operations
print("\nCreating S3 bucket")
create_s3_response = call_tool("executeCommand",
                              {"command": f"aws s3 mb {s3_path} --region {region}"})
print(f"create result: {create_s3_response}")

#### 9.3 Upload file from code interpreter by running command to the S3 Bucket(created above)

In [ ]:
print("\nUploading file to S3")
upload_to_s3_response = call_tool("executeCommand",
                                 {"command": f"aws s3 cp {files_to_create[0]['path']} {s3_path}"})
print(f"upload result: {upload_to_s3_response}")



#### 9.4 List files from the S3 bucket by running a command in the code interpreter

In [ ]:
print("\nListing files in S3")
list_s3_response = call_tool("executeCommand",
                            {"command": f"aws s3 ls {s3_path}"})
print(f"list result: {list_s3_response}")

## 10. Cleanup

Clean up resources by stopping the session and deleting the interpreter.

In [ ]:
# Cleanup
print("Cleaning up session and interpreter")
dp_client.stop_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id
)
print("Session stopped successfully")

cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
print("Interpreter deleted successfully")